In [3]:
!pip install category_encoders

DEPRECATION: nbmanips 2.0.1 has a non-standard dependency specifier cloudpickle>=1.6.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nbmanips or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: nbmanips 2.0.1 has a non-standard dependency specifier click>=7.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nbmanips or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: nbmanips 2.0.1 has a non-standard dependency specifier Pygments>=2.10.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nbmanips or contact the author to suggest that they release a version with

In [4]:
import pandas as pd
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


## Preprocessing

In [9]:
data=pd.read_csv("https://raw.githubusercontent.com/dharaneishvc/Botnet-Threat-Detection-ML/main/features_having_most_influence_on_Botnet_IoT.csv")
print("Length of data: ",len(data))
print("Dtype: ", data.dtypes)
print(data.describe())
print(data.head())


Length of data:  733705
Dtype:  pkSeqID                int64
proto                 object
saddr                 object
sport                 object
daddr                 object
dport                 object
seq                    int64
stddev               float64
N_IN_Conn_P_SrcIP      int64
min                  float64
state_number           int64
mean                 float64
N_IN_Conn_P_DstIP      int64
drate                float64
srate                float64
max                  float64
attack                 int64
category              object
subcategory           object
dtype: object
            pkSeqID            seq         stddev  N_IN_Conn_P_SrcIP  \
count  7.337050e+05  733705.000000  733705.000000      733705.000000   
mean   1.834472e+06  121412.819892       0.887894          82.492551   
std    1.058826e+06   75823.398840       0.804013          24.426145   
min    2.000000e+00       1.000000       0.000000           1.000000   
25%    9.172220e+05   54993.000000       0.

In [10]:
df = data.drop(['pkSeqID','subcategory','attack','dport','sport'],axis=1)

In [11]:
#encoder1 = ce.HashingEncoder(cols='saddr',n_components=6)

encoder1 = ce.BinaryEncoder(cols=['saddr'],return_df=True)
df = encoder1.fit_transform(df)

encoder2 = ce.BinaryEncoder(cols=['daddr'],return_df=True)
df = encoder2.fit_transform(df)

proto_encoded = pd.get_dummies(data=df['proto'],drop_first=True)

df = pd.concat([df,proto_encoded],axis=1)
df.drop('proto',axis=1,inplace=True)

print(df.dtypes)

saddr_0                int64
saddr_1                int64
saddr_2                int64
saddr_3                int64
saddr_4                int64
daddr_0                int64
daddr_1                int64
daddr_2                int64
daddr_3                int64
daddr_4                int64
daddr_5                int64
seq                    int64
stddev               float64
N_IN_Conn_P_SrcIP      int64
min                  float64
state_number           int64
mean                 float64
N_IN_Conn_P_DstIP      int64
drate                float64
srate                float64
max                  float64
category              object
icmp                   uint8
ipv6-icmp              uint8
tcp                    uint8
udp                    uint8
dtype: object


In [12]:
scaler = StandardScaler()
X = df.drop('category',axis=1)
y = df['category']
X = scaler.fit_transform(X)

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=109)

## Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

clf = LogisticRegression(random_state=0, multi_class='multinomial').fit(X_train, y_train)

predictions = clf.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test,predictions))
print("Classification report \n",classification_report(y_test,predictions,digits=6, zero_division=1))
print("Confusion Matrix:\n", confusion_matrix(y_test,predictions))

c:\Users\dharaneish\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy Score:  0.9853347053652354
Classification report 
                 precision    recall  f1-score   support

          DDoS   0.994369  0.984061  0.989188     76983
           DoS   0.979881  0.996145  0.987946     66152
        Normal   0.809524  0.944444  0.871795        18
Reconnaissance   0.888584  0.814226  0.849782      3585
         Theft   1.000000  0.000000  0.000000         3

      accuracy                       0.985335    146741
     macro avg   0.934472  0.747775  0.739742    146741
  weighted avg   0.985231  0.985335  0.985188    146741

Confusion Matrix:
 [[75756  1065     1   161     0]
 [   50 65897     2   203     0]
 [    0     0    17     1     0]
 [  379   286     1  2919     0]
 [    0     2     0     1     0]]


## Artificial Neural Networks

In [49]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = df['category'].values

y = ohe.fit_transform(y.reshape(-1,1)).toarray()

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=109)


In [50]:
model = Sequential()
model.add(Dense(16,input_dim=25,activation='relu'))
model.add(Dense(12,activation='relu'))
model.add(Dense(5,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=64)

y_pred = model.predict(X_test)

pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
test = list()
for i in range(len(y_test)):
      test.append(np.argmax(y_test[i]))


print('Accuracy:', accuracy_score(test,pred))

print("Classification report:\n",classification_report(test,pred,digits=6))
print("Confusion Matrix:\n", confusion_matrix(test, pred))


## Decision Tree Classifier with GridSearch

In [ ]:
X = df.drop('category',axis=1)
y = df['category']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=109)

In [ ]:
#Decision Tree Classifier with GridSearch
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dec = DecisionTreeClassifier()
params = {'max_depth':(1,2,3,10,100,1000)}
grid_search_dec = GridSearchCV(dec, params, cv=5, verbose=10, n_jobs=-1)
grid_search_dec.fit(X_train, Y_train)

print(grid_search_dec.best_params_)

y_pred = grid_search_dec.predict(X_test)

print('Accuracy:', accuracy_score(Y_test, y_pred))
print("Classification report:\n", classification_report(Y_test, y_pred, digits=6))
print("Confusion Matrix:\n", confusion_matrix(Y_test, y_pred))

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': (1, 2, 3, 10, 100, 1000)}, verbose=10)

## Random Forest Classifier with GridsearchCV

In [ ]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
randForest = RandomForestClassifier()
params = {'max_depth':(1,2,9,10),'n_estimators':(10,15,30)}
grid_search = GridSearchCV(randForest, params, cv=5, verbose=10)
grid_search.fit(X_train, Y_train)

y_pred = grid_search.predict(X_test)

print(grid_search.best_params_)

print('Accuracy:', accuracy_score(Y_test, y_pred))
print("Classification report: \n",classification_report(Y_test,y_pred,digits=6))
print("Confusion Matrix for RandomForest", confusion_matrix(Y_test, y_pred))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START max_depth=1, n_estimators=10...............................
[CV 1/5; 1/12] END max_depth=1, n_estimators=10;, score=0.788 total time=   2.9s
[CV 2/5; 1/12] START max_depth=1, n_estimators=10...............................
[CV 2/5; 1/12] END max_depth=1, n_estimators=10;, score=0.757 total time=   3.0s
[CV 3/5; 1/12] START max_depth=1, n_estimators=10...............................
[CV 3/5; 1/12] END max_depth=1, n_estimators=10;, score=0.829 total time=   3.6s
[CV 4/5; 1/12] START max_depth=1, n_estimators=10...............................
[CV 4/5; 1/12] END max_depth=1, n_estimators=10;, score=0.863 total time=   4.2s
[CV 5/5; 1/12] START max_depth=1, n_estimators=10...............................
[CV 5/5; 1/12] END max_depth=1, n_estimators=10;, score=0.845 total time=   3.5s
[CV 1/5; 2/12] START max_depth=1, n_estimators=15...............................
[CV 1/5; 2/12] END max_depth=1, n_estimators=15;

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': (1, 2, 9, 10),
                         'n_estimators': (10, 15, 30)},
             verbose=10)

## Notes

For evaluation metrics see https://scikit-learn.org/stable/modules/model_evaluation.html 

See Multiclass evaluation